In [1]:
import boto3
import sagemaker

s3_bucket = 'bucket'
s3_prefix = 'prefix'
pipeline_name = 'MyPipeline-2023-07-03-21-07-05'
region = 'eu-west-2'
role_arn =  sagemaker.get_execution_role()

eventbridge_client = boto3.client('events', region_name=region)
sagemaker_client = boto3.client('sagemaker', region_name=region)

# Get the pipeline ARN
pipeline_arn = sagemaker_client.list_pipelines(PipelineNamePrefix=pipeline_name)['PipelineSummaries'][0]['PipelineArn']

# Create the rule
rule_response = eventbridge_client.put_rule(
    Name='cmg_sagemaker_pipeline',
    EventPattern=f'{{"source": ["aws.s3"], "detail-type": ["ObjectCreated:*"], "detail": {{"bucket": {{"name": ["{s3_bucket}"]}}, "object": {{"key": [{{"prefix": "{s3_prefix}training"}}]}}}}}}',
    State='ENABLED',
    Description='Rule to trigger SageMaker pipeline when new file is added to S3 bucket',
    RoleArn=role_arn
)

# Add the target
target_response = eventbridge_client.put_targets(
    Rule='my-sagemaker-pipeline-trigger-rule',
    Targets=[
        {
            'Id': '1',
            'Arn': pipeline_arn,
            'RoleArn': role_arn,
            'Input': '{"PipelineExecutionDescription": "Started by EventBridge rule"}',
            'InputPath': '$.detail'
        }
    ]
)

print(f'Rule ARN: {rule_response["RuleArn"]}')


ClientError: An error occurred (AccessDeniedException) when calling the PutRule operation: User: arn:aws:sts::798861612849:assumed-role/AmazonSageMaker-ExecutionRole-20221020T100790/SageMaker is not authorized to perform: events:PutRule on resource: arn:aws:events:eu-west-2:798861612849:rule/cmg_sagemaker_pipeline because no identity-based policy allows the events:PutRule action